In [1]:
import lyricsgenius
import json
import pandas as pd
import nltk
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
nltk.download('punkt')
from nltk.lm.preprocessing import flatten
from nltk.tokenize import word_tokenize
from nltk.lm import KneserNeyInterpolated as KNI
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize, sent_tokenize 
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Forest_Gump\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Checking whether installed module works correctly, setting Eminem as the author
genius = lyricsgenius.Genius("jHJB8vqrbfVDIGM1KrETFx6s7EG9SGN3RGcmr63z7Gjgz8dYY4LizTL9rGupTAl1")
artist = genius.search_artist("Eminem", max_songs=100, sort="title")
genius.remove_section_headers = True
genius.verbose = False
print('')

Searching for songs by Eminem...

Song 1: "."
Song 2: "12 Days of Diss-Mas"
Song 3: "1-833-2GET-REV (REVIVAL Voicemail)"
Song 4: "1996 Underground Freestyle"
Song 5: "1997 Freestyle Live at Wetlands, NYC"
Song 6: "1997 Rap Olympics"
Song 7: "1999"
Song 8: "1999 Tim Westwood Freestyle"
Song 9: "2004 Tim Westwood Freestyle"
Song 10: "2012 Something From Nothing: Art of Rap Freestyle"
Song 11: "2.0 Boys"
Song 12: "25 To Life"
Song 13: "313"
Song 14: "3 a.m."
Song 15: "3 Verses"
Song 16: "4 Verses"
Song 17: "50 Ways"
Song 18: "8 Mile"
Song 19: "8 Mile: 313 Cypher (Chin Tiki Girls)"
Song 20: "8 Mile Background Music"
Song 21: "8 Mile: B-Rabbit vs Papa Doc"
Song 22: "8 Mile: B-Rabbit vs Supa Emcee"
Song 23: "8 Mile: Cheddar Bob Freestyle"
Song 24: "8 Mile: D’Phuzion vs B-Rabbit"
Song 25: "8 Mile: Lily’s Lullaby"
Song 26: "8 Mile: Lotto vs B-Rabbit"
Song 27: "8 Mile: Lyckety-Splyt vs B-Rabbit"
Song 28: "8 Mile: Marv Won vs B-Rabbit"
Song 29: "8 Mile: Maurice Grant vs B-Rabbit"
Song 30: "8 Mil

In [5]:
artist.save_lyrics()

Lyrics_Eminem.json already exists. Overwrite?
(y/n): y
Wrote `Lyrics_Eminem.json`


In [6]:
lyrics = 'Lyrics_Eminem.json'
with open(lyrics) as eminem_lyrics:
    messyvocab = json.load(eminem_lyrics)
dict_keys = messyvocab.keys()
print(dict_keys)

dict_keys(['alternate_names', 'api_path', 'description', 'facebook_name', 'followers_count', 'header_image_url', 'id', 'image_url', 'instagram_name', 'is_meme_verified', 'is_verified', 'name', 'translation_artist', 'twitter_name', 'url', 'current_user_metadata', 'iq', 'description_annotation', 'user', 'songs'])


In [7]:
vocab = messyvocab.get('songs')
lyrics_vocab = []
for i in range(0,len(vocab)):
    index = vocab[i]
    song = index.get('lyrics')
    lyrics_vocab.append(song)
lyricsdf = pd.DataFrame(lyrics_vocab, columns=['Song lyrics'])
lyricsdf.head(3)

,Song lyrics
0,/
1,"1. ""Hit 'Em Up"" by 2Pac (Ft. Outlawz)\n2. ""Dis..."
2,[Automated Voice]\nThank you for your interest...


In [332]:
lyricsdf = lyricsdf.dropna()
print(len(lyricsdf))

99


In [333]:
punctuation = ['!','"','#','$','%','&','(',')','*','+',',','-','/',':',';','<','=','>','?','@','[','\\',']',
 '^','_','`','{','|','}','~', '46', '...']

In [334]:
for i in range(0,len(lyricsdf)):
    stringlyric = lyricsdf.iloc[i,0]
    stringlyric.replace("\n\n\n\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n", "\n")
    stringlyric.replace("\n\n", "\n")
    stringlyric.replace("\\n", "\n")
    stringlyric.replace("\\", "\n")
    for x in punctuation: 
        stringlyric = stringlyric.replace(x, '')
        lyricsdf.iloc[i,0] = stringlyric

## Perplexity problems

In [335]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(lyricsdf, test_size=0.7, random_state=11)

In [336]:
tr = str(train['Song lyrics'])
tr = tr.lower()
splitted = tr.split("\n")

In [337]:
#change discount later
paddedLine = [list(pad_both_ends(word_tokenize(tr), n=2))]
trainset, vocab = padded_everygram_pipeline(2, paddedLine)
lm = KNI(2,discount=0.1);
lm.fit(trainset,vocab)

In [363]:
test1 = str(test['Song lyrics'])
test1=test1.lower()
test1.split("\n")
tok = nltk.word_tokenize(test1)
bigr = nltk.bigrams(tok)
print(list(bigr))

[('46', 'intro\\nwow'), ('intro\\nwow', 'baby\\n\\nchorus\\nthe'), ('baby\\n\\nchorus\\nthe', 'way'), ('way', 'you'), ('you', 'shake'), ('shake', 'i'), ('i', '...'), ('...', '49'), ('49', 'verse'), ('verse', '1\\none'), ('1\\none', 'thousand'), ('thousand', 'different'), ('different', 'houses'), ('houses', 'and'), ('and', 'mün'), ('mün', '...'), ('...', '22'), ('22', 'brabbit'), ('brabbit', 'eminem\\ncheddar'), ('eminem\\ncheddar', 'i'), ('i', 'can'), ('can', 'rip'), ('rip', 'you'), ('you', 'to'), ('to', 'a'), ('a', 'shr'), ('shr', '...'), ('...', '58'), ('58', 'bust'), ('bust', 'it\\nthe'), ('it\\nthe', 'final'), ('final', 'problem'), ('problem', 'is'), ('is', 'solved'), ('solved', 'by'), ('by', 'throwi'), ('throwi', '...'), ('...', '41'), ('41', 'all'), ('all', 'i'), ('i', 'got'), ('got', 'to'), ('to', 'do'), ('do', 'is'), ('is', 'throw'), ('throw', 'any'), ('any', 'word'), ('word', 'at'), ('at', 'you'), ('you', 'you'), ('you', 'c'), ('c', '...'), ('...', '...'), ('...', '14'), ('14'

In [364]:
#больше похоже на пашу техника, конечно
print(lm.generate(20, random_seed=3))

['about', 'to', 'kick', '...', '67', 'intro', 'dj', 'green', 'lantern', 'eminem\\nfucker\\nyeah\\n', '...', '45', 'verse', '1', 'eminem\\ncame', 'to', 'the', 'world', 'at', 'a']


In [365]:
lm.perplexity(bigr)

ZeroDivisionError: division by zero

Heavily based on:
    https://www.kaggle.com/alvations/n-gram-language-model-with-nltk